In [1]:
import importlib
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.utils import gen_batches
import numpy as np
from Code.SGDPLS import SGDPLS
from Code.OLPLS import OLPLS
from Code.CIPLS import CIPLS
from Code.SIMPLS import SIMPLS
from Code.IPLS import IPLS
from Code.PLS1 import IPLS1
import h5py
# ----------
from sklearn.cross_decomposition import PLSRegression
from pytictoc import TicToc
tim=TicToc()

In [2]:
dataset='CTSlices'
CT_dat = np.genfromtxt("data/CTSlices/slice_localization_data.csv",
                       delimiter=",",
                       skip_header=1)
X_train = CT_dat[:,1:-1]
y_train = CT_dat[:,-1]

In [3]:
n_train = X_train.shape[0]
batch_size = 100
batch_num = int(n_train/batch_size)
print(f"n_train: {n_train:d}; batch_num: {batch_num:d}, batch_size: {batch_size:d}")

# batches = gen_batches(n_train, batch_size)
train_ind = slice((None, n_train))
trainset = [ slice((i+1)*batch_size) for i in range(batch_num)]
samples = [(i+1)*batch_size for i in range(batch_num)]

n_train: 53500; batch_num: 535, batch_size: 100


In [4]:
def RunExp(method, PLS, case, save):
    tim = TicToc()
    timlist=np.zeros((2, len(trainset)))
    timlist[0] = np.array(samples)
    if (case==1):
        for i,s in enumerate(gen_batches(n_train, batch_size)):
            tim.tic()
            PLS.fit(X_train[s], y_train[s])
            timlist[1,i] = tim.tocvalue()
    else:
        for i,s in enumerate(trainset):
            tim.tic()
            PLS.fit(X_train[s], y_train[s])
            timlist[1,i] = tim.tocvalue()


    if(method != 'NIPALS'):
        PLS._comp_coef()
        coef  = PLS.coef_
    else:
        coef  = PLS.coef_.ravel()

    if (save==1):
        with h5py.File(f"./Results/PLS1_CTSlices_{method}.h5", "w") as f:
            f_coef = f.create_dataset('coef', data=coef, maxshape=coef.shape, chunks=True)
            f_tim = f.create_dataset('timer', data=timlist, maxshape=timlist.shape, chunks=True)


In [5]:
from sklearn.cross_decomposition import PLSRegression

save=1
tim.tic()
PLS = PLSRegression(n_components=9, scale=False)
RunExp('NIPALS', PLS, 0, save)
print("elapsed time:", tim.tocvalue() )

elapsed time: 316.44085395429283


In [6]:
import Code.OLPLS
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

save=1
tim.tic()
PLS = OLPLS(n_components=9,amnesic=0.99,mu=1e-5)
RunExp('OLPLS', PLS, 1, save)
print("elapsed time:", tim.tocvalue() )

elapsed time: 106.120006125886


In [7]:
import Code.PLS1
importlib.reload(Code.PLS1)
from Code.PLS1 import PLS1

save=1
tim.tic()
PLS= PLS1(n_components=9)
RunExp('PLS1', PLS, 0, save)
print("elapsed time:", tim.tocvalue() )

elapsed time: 74.71643773140386


In [14]:
import Code.PLS1
importlib.reload(Code.PLS1)
from Code.PLS1 import IPLS1

save=1
tim.tic()
PLS= IPLS1(n_components=9)
RunExp('IPLS1', PLS, 1, save)
print("elapsed time:", tim.tocvalue() )

elapsed time: 2.0642851940356195


In [15]:
import Code.CIPLS
importlib.reload(Code.CIPLS)
from Code.CIPLS import CIPLS

save=1
tim.tic()
PLS= CIPLS(n_components=9)
RunExp('CIPLS', PLS, 1, save)
print("elapsed time:", tim.tocvalue() )

/video/data/tingan/GitRep/IPLSreview/Code/CIPLS.py:102: RuntimeWarning: overflow encountered in multiply
  x_weights_[c]   += (u * l)
/home/tingan/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input contains infinity or a value too large for dtype('float64').

In [10]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

save=1
tim.tic()
PLS = SIMPLS(n_components=8)
RunExp('SIMPLS', PLS, 0, save)
print("elapsed time:", tim.tocvalue() )

elapsed time: 69.92439557006583


In [16]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS

save=1
tim.tic()
PLS= ISIMPLS(n_components=8)
RunExp('ISIMPLS', PLS, 1, save)
print("elapsed time:", tim.tocvalue() )

elapsed time: 2.215983693022281
